# 10K Analysis - Sub Question Query Engine

In [2]:
from llama_index import SimpleDirectoryReader, LLMPredictor, ServiceContext, GPTVectorStoreIndex
from llama_index.response.pprint_utils import pprint_response
from langchain import OpenAI

/Users/suo/miniconda3/envs/llama/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


configure LLM

In [3]:
llm_predictor = LLMPredictor(llm=OpenAI(temperature=0, model_name="text-davinci-003", max_tokens=-1, streaming=True))
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor)

load data 

In [4]:
lyft_docs = SimpleDirectoryReader(input_files=["../data/10k/lyft_2021.pdf"]).load_data()
uber_docs = SimpleDirectoryReader(input_files=["../data/10k/uber_2021.pdf"]).load_data()

build indices

In [5]:
lyft_index = GPTVectorStoreIndex.from_documents(lyft_docs)

In [8]:
uber_index = GPTVectorStoreIndex.from_documents(uber_docs)

configure query engine

In [6]:
lyft_engine = lyft_index.as_query_engine(similarity_top_k=3)

In [9]:
uber_engine = uber_index.as_query_engine(similarity_top_k=3)

# build sub question query engine

In [11]:
from llama_index import GPTVectorStoreIndex, SimpleDirectoryReader
from llama_index.tools import QueryEngineTool, ToolMetadata
from llama_index.query_engine.sub_question_query_engine import SubQuestionQueryEngine

In [15]:
query_engine_tools = [
    QueryEngineTool(query_engine=lyft_engine, metadata=ToolMetadata(name='lyft_10k', description='Provides information about Lyft financials for year 2021')),
    QueryEngineTool(query_engine=uber_engine, metadata=ToolMetadata(name='uber_10k', description='Provides information about Uber financials for year 2021')),
]

s_engine = SubQuestionQueryEngine.from_defaults(query_engine_tools=query_engine_tools)

In [22]:
response = s_engine.query('How does Uber revenue growth and EBITDA growth compare to Lyft in 2021')

Generated 4 sub questions.
[uber_10k] Q: What is the revenue growth of Uber
[uber_10k] A: Technologies, Inc. from 2020 to 2021?

The revenue growth of Uber Technologies, Inc. from 2020 to 2021 is 57%, with revenue increasing from $11.1 billion to $17.5 billion.
[uber_10k] Q: What is the EbITDA of Uber
[uber_10k] A: Technologies, Inc. for 2021?

The Adjusted EBITDA of Uber Technologies, Inc. for 2021 was -$774 million.
[lyft_10k] Q: What is the revenue growth of Lyft
[lyft_10k] A: from 2019 to 2021?

The revenue growth of Lyft from 2019 to 2021 is approximately 11.2%.
[lyft_10k] Q: What is the EbITDA of Lyft
[lyft_10k] A: 
The EbITDA of Lyft cannot be determined from the given context information.


In [25]:
print(response)


In 2021, Uber Technologies, Inc. had a revenue growth of 57%, while Lyft had a revenue growth of 11.2%. Uber Technologies, Inc. had an Adjusted EBITDA of -$774 million, while Lyft's EBITDA cannot be determined from the given context information.
